In [1]:
print("hi")

hi


In [3]:
import re
import time
import pandas as pd
from datetime import datetime
import os

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ------------------ Launch & open Redbus ------------------
driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.redbus.in/")
wait = WebDriverWait(driver, 25)

# ------------------ Helper to close any popup/dialog ------------------
def close_popup_if_present():
    """Close any popup or dialog that might be blocking interactions"""
    try:
        close_btn = driver.find_element(By.XPATH, "//button[contains(@class,'actionButton') and contains(@class,'action__cd46a7')]//i[contains(@class,'icon-close')]")
        close_btn.click()
        time.sleep(0.5)
        print("Closed popup dialog")
    except:
        pass

# ------------------ Open TGSRTC ------------------
tgsrtc_transport = wait.until(EC.element_to_be_clickable((By.XPATH, "//h3[text()='TGSRTC']")))
tgsrtc_transport.click()

# Close any popup after clicking TGSRTC
time.sleep(1)
close_popup_if_present()

# ------------------ Get all route links ------------------
ROUTE_XPATH = "//div[contains(@class,'route_details')]//a[contains(@class,'route')]"
wait.until(EC.presence_of_element_located((By.XPATH, ROUTE_XPATH)))

# Get total number of routes available
all_route_elements = driver.find_elements(By.XPATH, ROUTE_XPATH)
total_routes = len(all_route_elements)
print(f"\n{'='*60}")
print(f"Total routes found: {total_routes}")
print(f"{'='*60}\n")

# ------------------ XPaths for bus cards and details ------------------
CARD_XPATH = "//li[contains(@class,'tupleWrapper') and @role='button']"
BUS_NAME_X = ".//div[contains(@class,'travelsName')]"
BUS_TYPE_X = ".//p[contains(@class,'busType')]"
PRICE_X_OPTIONS = [
    ".//p[contains(@class,'finalFare')]",
    ".//div[contains(@class,'fareDetails')]//p[contains(@class,'fare')]",
    ".//span[contains(@class,'fare')]"
]

# ------------------ XPaths on details/reviews pages ------------------
READ_ALL_REVIEWS_BTN_X = "//button[contains(@aria-label,'Read all') and contains(., 'Read all')]"
RATING_CONTAINER_X = "//div[contains(@class,'container___39506b')]"
REVIEW_CARD_X   = "//div[contains(@class,'customerReviewCont')]"
REVIEW_NAME_X   = ".//h4[contains(@class,'title')]"
REVIEW_DATE_X   = ".//div[contains(@class,'date')]"
REVIEW_STARS_X  = ".//div[contains(@class,'chip') and contains(@aria-label,'star')]"
REVIEW_TEXT_X   = ".//div[contains(@class,'reviewCmt')]"

# ------------------ Helper functions ------------------
def load_bus_cards_until(min_needed=2, max_idle_rounds=3, pause=2.5):
    same_rounds = 0
    prev_count = 0
    while True:
        cards_now = driver.find_elements(By.XPATH, CARD_XPATH)
        if cards_now:
            driver.execute_script("arguments[0].scrollIntoView({block: 'end'});", cards_now[-1])
        time.sleep(pause)
        new_count = len(driver.find_elements(By.XPATH, CARD_XPATH))
        if new_count >= min_needed and same_rounds >= 1:
            break
        if new_count == prev_count:
            same_rounds += 1
        else:
            same_rounds = 0
        prev_count = new_count
        if same_rounds >= max_idle_rounds:
            break

def parse_star_from_aria(aria_label: str) -> int:
    if not aria_label:
        return 0
    m = re.search(r"(\d+)\s*star", aria_label.lower())
    return int(m.group(1)) if m else 0

def to_mm_dd_yyyy(date_text: str) -> str:
    date_text = date_text.strip()
    for fmt in ("%d %b %Y", "%d %B %Y"):
        try:
            dt = datetime.strptime(date_text, fmt)
            return dt.strftime("%m-%d-%Y")
        except ValueError:
            continue
    return date_text

def extract_rating_categories():
    ratings = {}
    try:
        time.sleep(2)
        
        # Try multiple XPath strategies to find rating containers
        rating_containers = driver.find_elements(By.XPATH, "//div[contains(@class,'container___') and contains(@class,'undefined')]")
        
        if not rating_containers:
            rating_containers = driver.find_elements(By.XPATH, "//div[@role='listitem']//div[contains(@class,'container')]")
        
        if not rating_containers:
            rating_containers = driver.find_elements(By.XPATH, "//div[contains(@class,'label__')]")
        
        print(f"Found {len(rating_containers)} rating containers")
        
        for container in rating_containers:
            try:
                # Get the full text of the container
                container_text = container.text.strip()
                
                if not container_text:
                    continue
                
                # Try to find label element with class label__f10328
                label_elem = None
                try:
                    label_elem = container.find_element(By.XPATH, ".//div[contains(@class,'label__')]")
                except:
                    try:
                        label_elem = container.find_element(By.XPATH, ".//div[@class='label__f10328']")
                    except:
                        pass
                
                if label_elem:
                    label_text = label_elem.text.strip()
                else:
                    label_text = container_text
                
                print(f"Processing label: {label_text}")
                
                # Extract category name and rating value from text like "Punctuality (1249)"
                match = re.match(r"(.+?)\s*\((\d+)\)", label_text)
                if match:
                    category_name = match.group(1).strip()
                    rating_value = match.group(2).strip()
                    
                    # Normalize category names
                    if "Staff behavior" in category_name or "Staff behaviour" in category_name:
                        category_name = "Staff Behaviour"
                    elif "Seat / Sleep Comfort" in category_name or "Seat comfort" in category_name:
                        category_name = "Seat comfort"
                    elif "Punctuality" in category_name:
                        category_name = "Punctuality"
                    elif "Cleanliness" in category_name:
                        category_name = "Cleanliness"
                    elif "Driving" in category_name:
                        category_name = "Driving"
                    elif "AC" in category_name:
                        category_name = "AC"
                    elif "Rest stop hygiene" in category_name:
                        category_name = "Rest stop hygiene"
                    elif "Live tracking" in category_name:
                        category_name = "Live tracking"
                    
                    ratings[category_name] = rating_value
                    print(f"Captured: {category_name} = {rating_value}")
            except Exception as e:
                print(f"Error processing container: {e}")
                continue
    except Exception as e:
        print(f"Error extracting rating categories: {e}")
    
    # Define expected categories with default values
    expected_categories = {
        "Punctuality": "N/A", 
        "Cleanliness": "N/A", 
        "Staff Behaviour": "N/A",
        "Driving": "N/A", 
        "AC": "N/A", 
        "Rest stop hygiene": "N/A",
        "Seat comfort": "N/A", 
        "Live tracking": "N/A"
    }
    
    # Fill in missing categories with N/A
    for cat, default in expected_categories.items():
        if cat not in ratings:
            ratings[cat] = default
    
    print(f"Final ratings: {ratings}")
    return ratings

def scroll_reviews_to_load_n(target_reviews='all', max_scrolls=1000, pause=1.0):
    goal = None
    if isinstance(target_reviews, int) and target_reviews > 0:
        goal = target_reviews
    else:
        try:
            hdr = driver.find_elements(By.XPATH, "//h3[contains(., 'reviews')]")
            if hdr:
                m = re.search(r"(\d+)\s+reviews", hdr[0].text.strip().lower())
                if m:
                    goal = int(m.group(1))
        except:
            goal = None

    scrolls = 0
    same_count_rounds = 0
    prev_count = 0
    reviews_container = None
    
    try:
        cards_now = driver.find_elements(By.XPATH, REVIEW_CARD_X)
        if cards_now:
            first_card = cards_now[0]
            reviews_container = driver.execute_script("""
                var el = arguments[0];
                while (el) {
                  try {
                    var cs = getComputedStyle(el);
                    if ((cs.overflowY === 'auto' || cs.overflowY === 'scroll') && el.scrollHeight > el.clientHeight) {
                      return el;
                    }
                  } catch (e) {}
                  el = el.parentElement;
                }
                return null;
            """, first_card)
    except:
        reviews_container = None

    def _click_more_button_if_present():
        btn_xpath = (
            "//button["
            "contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'load more') or "
            "contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'show more') or "
            "contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'view more') or "
            "contains(translate(@aria-label, 'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'load more') or "
            "contains(translate(@aria-label, 'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'show more') or "
            "contains(translate(@aria-label, 'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'view more')"
            "]"
        )
        try:
            btns = driver.find_elements(By.XPATH, btn_xpath)
            if btns:
                btn = btns[0]
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
                time.sleep(0.2)
                try:
                    btn.click()
                except:
                    driver.execute_script("arguments[0].click();", btn)
                time.sleep(1.0)
                return True
        except:
            pass
        return False

    while scrolls < max_scrolls:
        cards = driver.find_elements(By.XPATH, REVIEW_CARD_X)
        current_count = len(cards)

        if goal is not None and current_count >= goal:
            print(f"  Loaded {current_count} reviews (goal {goal}).")
            break

        clicked = _click_more_button_if_present()

        if not clicked:
            if current_count:
                try:
                    driver.execute_script("arguments[0].scrollIntoView({block:'end'});", cards[-1])
                except:
                    pass

            if reviews_container:
                try:
                    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollTop + 1600;", reviews_container)
                except:
                    reviews_container = None
            else:
                driver.execute_script("window.scrollBy(0, 1600);")

            time.sleep(pause)

            for _ in range(3):
                if reviews_container:
                    try:
                        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollTop + 400;", reviews_container)
                    except:
                        break
                else:
                    driver.execute_script("window.scrollBy(0, 400);")
                time.sleep(0.25)

        new_count = len(driver.find_elements(By.XPATH, REVIEW_CARD_X))
        if new_count == prev_count:
            same_count_rounds += 1
            if same_count_rounds >= 12:
                print(f"  All available reviews loaded: {new_count} (no more).")
                break
        else:
            same_count_rounds = 0
            if scrolls % 5 == 0:
                print(f"  Progress: {new_count} reviews loaded...")

        prev_count = new_count
        scrolls += 1

# ------------------ Main data collection ------------------
rows = []

# Check if there's an existing CSV to resume from
processed_routes = set()
resume_mode = False

if os.path.exists("redbus_tgsrtc1.csv"):
    try:
        existing_df = pd.read_csv("redbus_tgsrtc1.csv", encoding="utf-8-sig")
        if not existing_df.empty:
            # Load existing data
            rows = existing_df.to_dict('records')
            # Get list of already processed routes
            processed_routes = set(existing_df['Route'].unique())
            resume_mode = True
            print(f"\n{'='*60}")
            print(f"RESUME MODE: Found existing data")
            print(f"Already processed routes: {len(processed_routes)}")
            print(f"Routes processed: {list(processed_routes)}")
            print(f"Existing rows: {len(rows)}")
            print(f"Will skip already processed routes")
            print(f"{'='*60}\n")
    except Exception as e:
        print(f"Could not load existing CSV: {e}")
        print("Starting fresh...\n")

if not resume_mode:
    print(f"\n{'='*60}")
    print(f"STARTING FRESH SCRAPE")
    print(f"Will save to: redbus_tgsrtc1.csv")
    print(f"{'='*60}\n")

# Loop through routes starting from route 1 (index starts at 1)
START_ROUTE = 1

# Store route information before starting
route_links_info = []
all_route_elements = driver.find_elements(By.XPATH, ROUTE_XPATH)
for idx, route_el in enumerate(all_route_elements, start=1):
    if idx >= START_ROUTE:
        route_links_info.append({
            'index': idx,
            'text': route_el.text.strip(),
            'href': route_el.get_attribute('href')
        })

print(f"Routes to process (from route {START_ROUTE} onwards):")
for info in route_links_info:
    print(f"  Route {info['index']}: {info['text']}")
print()

# Process each route using direct URLs
for route_info in route_links_info:
    route_idx = route_info['index']
    route_text = route_info['text']
    route_url = route_info['href']
    
    print(f"\n{'#'*60}")
    print(f"# PROCESSING ROUTE {route_idx} of {total_routes}")
    print(f"# Route: {route_text}")
    print(f"{'#'*60}\n")
    
    # Skip if already processed (for resume capability)
    if route_text in processed_routes:
        print(f"⏭️  SKIPPING: Route '{route_text}' already processed")
        continue
    
    # Navigate directly to the route URL
    driver.get(route_url)
    time.sleep(2)
    close_popup_if_present()
    
    # Wait for bus cards to load with error handling
    try:
        wait.until(EC.presence_of_element_located((By.XPATH, CARD_XPATH)))
    except Exception as e:
        print(f"⚠️ WARNING: No bus cards found for route '{route_text}'")
        print(f"Error: {e}")
        print(f"Skipping this route and moving to next one...\n")
        continue
    
    # Load 100 buses for this route
    load_bus_cards_until(min_needed=100, max_idle_rounds=5, pause=3)
    
    # Get buses for this route (limit to 100)
    cards = driver.find_elements(By.XPATH, CARD_XPATH)
    
    # Check if any buses were found
    if len(cards) == 0:
        print(f"⚠️ WARNING: No buses available for route '{route_text}'")
        print(f"Skipping this route and moving to next one...\n")
        continue
    
    cards_to_process = min(100, len(cards))
    print(f"Total buses found in this route: {len(cards)}, will process: {cards_to_process}")
    
    # Process each bus in this route
    for idx in range(cards_to_process):
        print(f"\n--- Processing Bus {idx + 1} of {cards_to_process} ---")
        
        close_popup_if_present()
        
        # Re-locate cards each time and verify the card still exists
        cards = driver.find_elements(By.XPATH, CARD_XPATH)
        
        # If we don't have enough cards, try to reload them
        if idx >= len(cards):
            print(f"Card index {idx} not found, reloading bus list...")
            load_bus_cards_until(min_needed=cards_to_process, max_idle_rounds=5, pause=3)
            cards = driver.find_elements(By.XPATH, CARD_XPATH)
            
            # If still not enough cards, skip this bus
            if idx >= len(cards):
                print(f"⚠️ WARNING: Bus {idx + 1} not available, skipping...")
                continue
        
        card = cards[idx]

        # Extract list-page info
        try:
            bus_name = card.find_element(By.XPATH, BUS_NAME_X).text.strip()
        except:
            bus_name = ""
        
        try:
            bus_type = card.find_element(By.XPATH, BUS_TYPE_X).text.strip()
        except:
            bus_type = ""
        
        price = ""
        for price_xpath in PRICE_X_OPTIONS:
            try:
                price = card.find_element(By.XPATH, price_xpath).text.strip()
                if price:
                    break
            except:
                continue

        print(f"Bus Name: {bus_name}, Type: {bus_type}, Price: {price}")

        # Open bus details page
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", card)
        time.sleep(0.5)
        close_popup_if_present()
        
        try:
            card.click()
        except:
            try:
                card.send_keys(Keys.ENTER)
            except:
                driver.execute_script("arguments[0].click();", card)

        time.sleep(2)
        close_popup_if_present()
        
        # Extract rating categories
        rating_categories = extract_rating_categories()
        print(f"Rating Categories: {rating_categories}")

        # Try to open reviews page
        try:
            close_popup_if_present()
            read_all_btn = wait.until(EC.element_to_be_clickable((By.XPATH, READ_ALL_REVIEWS_BTN_X)))
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", read_all_btn)
            time.sleep(0.5)
            
            try:
                read_all_btn.click()
            except:
                driver.execute_script("arguments[0].click();", read_all_btn)
                
        except Exception as e:
            print(f"No reviews button found for this bus: {e}")
            driver.back()
            wait.until(EC.presence_of_element_located((By.XPATH, CARD_XPATH)))
            load_bus_cards_until(min_needed=cards_to_process)
            continue

        # Load all reviews
        try:
            wait.until(EC.presence_of_element_located((By.XPATH, REVIEW_CARD_X)))
            scroll_reviews_to_load_n(target_reviews='all')
        except:
            pass

        # Parse all review cards
        review_cards = driver.find_elements(By.XPATH, REVIEW_CARD_X)
        reviews_to_fetch = len(review_cards)
        print(f"Found {len(review_cards)} reviews, fetching all")

        for r_idx in range(reviews_to_fetch):
            rc = review_cards[r_idx]
            
            try:
                passenger = rc.find_element(By.XPATH, REVIEW_NAME_X).text.strip()
            except:
                passenger = ""

            try:
                raw_date = rc.find_element(By.XPATH, REVIEW_DATE_X).text.strip()
                review_date = to_mm_dd_yyyy(raw_date)
            except:
                review_date = ""

            try:
                aria = rc.find_element(By.XPATH, REVIEW_STARS_X).get_attribute("aria-label") or ""
                stars = parse_star_from_aria(aria)
            except:
                stars = 0

            try:
                review_text = rc.find_element(By.XPATH, REVIEW_TEXT_X).text.strip()
            except:
                review_text = ""

            rows.append({
                "Route": route_text,
                "Bus Name": bus_name,
                "Bus Type": bus_type,
                "Price": price,
                "Punctuality": rating_categories.get("Punctuality", "N/A"),
                "Cleanliness": rating_categories.get("Cleanliness", "N/A"),
                "Staff Behaviour": rating_categories.get("Staff Behaviour", "N/A"),
                "Driving": rating_categories.get("Driving", "N/A"),
                "AC": rating_categories.get("AC", "N/A"),
                "Rest stop hygiene": rating_categories.get("Rest stop hygiene", "N/A"),
                "Seat comfort": rating_categories.get("Seat comfort", "N/A"),
                "Live tracking": rating_categories.get("Live tracking", "N/A"),
                "Name of passenger": passenger,
                "Date of Review": review_date,
                "Star Rating": stars,
                "Review Text": review_text,
            })

        # Go back to bus list
        driver.back()
        time.sleep(0.5)
        driver.back()
        
        # Wait for bus cards to reload
        try:
            wait.until(EC.presence_of_element_located((By.XPATH, CARD_XPATH)))
            time.sleep(1)  # Extra wait for DOM to stabilize
            load_bus_cards_until(min_needed=cards_to_process, max_idle_rounds=5, pause=3)
        except Exception as e:
            print(f"Warning: Error reloading bus list: {e}")
            # Try scrolling to trigger lazy loading
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
        
        close_popup_if_present()
    
    print(f"\nCompleted Route {route_idx}: {route_text}")
    print(f"Total rows collected so far: {len(rows)}")
    
    # Save after each route
    df = pd.DataFrame(rows)
    df.to_csv("redbus_tgsrtc1.csv", index=False, encoding="utf-8-sig")
    print(f"✓ Saved checkpoint: {len(rows)} rows to redbus_tgsrtc1.csv")
    print(f"{'='*60}\n")

# ------------------ Final Summary ------------------
print(f"\n{'='*60}")
print(f"SCRAPING COMPLETED!")
print(f"Total routes processed: {total_routes - START_ROUTE + 1}")
print(f"Total rows collected: {len(rows)}")
print(f"Final file: redbus_tgsrtc1.csv")
print(f"{'='*60}")

driver.quit()


Total routes found: 11


RESUME MODE: Found existing data
Already processed routes: 1
Routes processed: ['Hyderabad to Vijayawada']
Existing rows: 6629
Will skip already processed routes

Routes to process (from route 1 onwards):
  Route 1: Hyderabad to Vijayawada
  Route 2: Khammam to Hyderabad
  Route 3: Hyderabad to Srisailam
  Route 4: Hyderabad to Khammam
  Route 5: Karimnagar to Hyderabad
  Route 6: Hyderabad to Ongole
  Route 7: Kothagudem to Hyderabad
  Route 8: Hyderabad to Adilabad
  Route 9: Guntur (Andhra Pradesh) to Hyderabad
  Route 10: Jagityal to Hyderabad
  Route 11: Godavarikhani to Hyderabad


############################################################
# PROCESSING ROUTE 1 of 11
# Route: Hyderabad to Vijayawada
############################################################

⏭️  SKIPPING: Route 'Hyderabad to Vijayawada' already processed

############################################################
# PROCESSING ROUTE 2 of 11
# Route: Khammam to Hyderabad
#############

In [2]:
import pandas as pd
df_tg = pd.read_csv(r"D:\prakash\redbus_scraper\redbus_tgsrtc1.csv")

In [4]:
df_tg.shape

(10014, 16)

In [5]:
df_tg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10014 entries, 0 to 10013
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Route              10014 non-null  object 
 1   Bus Name           10014 non-null  object 
 2   Bus Type           10014 non-null  object 
 3   Price              10014 non-null  object 
 4   Punctuality        9997 non-null   float64
 5   Cleanliness        9999 non-null   float64
 6   Staff Behaviour    9971 non-null   float64
 7   Driving            9950 non-null   float64
 8   AC                 9370 non-null   float64
 9   Rest stop hygiene  9956 non-null   float64
 10  Seat comfort       9950 non-null   float64
 11  Live tracking      9953 non-null   float64
 12  Name of passenger  10013 non-null  object 
 13  Date of Review     10010 non-null  object 
 14  Star Rating        10014 non-null  int64  
 15  Review Text        10006 non-null  object 
dtypes: float64(8), int64(1

In [6]:
for col in df_tg.columns:
    print(f"Column: {col}")
    print(df_tg[col].unique())
    print("-" * 40)


Column: Route
['Hyderabad to Vijayawada' 'Hyderabad to Srisailam'
 'Karimnagar to Hyderabad' 'Kothagudem to Hyderabad'
 'Hyderabad to Adilabad' 'Guntur (Andhra Pradesh) to Hyderabad']
----------------------------------------
Column: Bus Name
['IntrCity SmartBus' 'FRESHBUS' 'Sai RK Travels' 'zingbus plus' 'NueGo'
 'FlixBus' 'Sri Sanvi Tours and Travels' 'Vikram Travels'
 'Tirumala Travels' 'Sri Durga Malleswari Travels' 'V Kaveri Travels'
 'PRK Travels' 'Tirumala Xpress' 'Svkdt travels'
 'Holidays Tours and Travels' 'BSR Tours And Travels'
 'Go Tour Travels and Holidays' 'Sri Sai Anjana Tours and Travels'
 'LVP Travels' 'KARTHIKEYA TOURS AND TRAVELS' 'Sri Shivanjaneya Travels'
 'LG BUS' 'Mythri Tours And Travels' 'SLGT Travels' '7Hills roadways'
 'Naveen Tours and Travels' 'VSR Tours and Travels'
 'Bharathi Tours & Travels' 'YASHWANTH BUS' 'SAIRAJ TRAVELS'
 'Jaya Rajeswari Travels' 'Morning Star Travels'
 'Shyamoli Paribahan Pvt Ltd' 'A1 Yellow Tours and Travels'
 'Sai Krishna Bus Servi

In [7]:
df_tg['Price'] = (
    df_tg['Price']
    .astype(str)                    # ensure it's a string
    .str.replace('₹', '', regex=False)  # remove rupee symbol
    .str.replace(',', '', regex=False)  # remove commas
    .str.strip()                    # trim spaces
)

# convert to numeric
df_tg['Price'] = pd.to_numeric(df_tg['Price'], errors='coerce')


In [8]:
df_tg.head(5)

,Route,Bus Name,Bus Type,Price,Punctuality,Cleanliness,Staff Behaviour,Driving,AC,Rest stop hygiene,Seat comfort,Live tracking,Name of passenger,Date of Review,Star Rating,Review Text
0,Hyderabad to Vijayawada,IntrCity SmartBus,A/C Seater / Sleeper (2+1),279,1249.0,1056.0,1111.0,1000.0,882.0,527.0,1078.0,813.0,Rama Gayatri,11-17-2025,5,Very good services.
1,Hyderabad to Vijayawada,IntrCity SmartBus,A/C Seater / Sleeper (2+1),279,1249.0,1056.0,1111.0,1000.0,882.0,527.0,1078.0,813.0,asad,11-17-2025,5,Good journey.
2,Hyderabad to Vijayawada,IntrCity SmartBus,A/C Seater / Sleeper (2+1),279,1249.0,1056.0,1111.0,1000.0,882.0,527.0,1078.0,813.0,Parvesh M,11-16-2025,5,Good.
3,Hyderabad to Vijayawada,IntrCity SmartBus,A/C Seater / Sleeper (2+1),279,1249.0,1056.0,1111.0,1000.0,882.0,527.0,1078.0,813.0,Akula Naresh,11-15-2025,5,Good experience.
4,Hyderabad to Vijayawada,IntrCity SmartBus,A/C Seater / Sleeper (2+1),279,1249.0,1056.0,1111.0,1000.0,882.0,527.0,1078.0,813.0,chetan Bagde,11-15-2025,5,Ok.


In [9]:

df_tg['Date of Review'] = pd.to_datetime(df_tg['Date of Review'], format='%m-%d-%Y', errors='coerce')



In [10]:
df_tg.isnull().sum()

Route                  0
Bus Name               0
Bus Type               0
Price                  0
Punctuality           17
Cleanliness           15
Staff Behaviour       43
Driving               64
AC                   644
Rest stop hygiene     58
Seat comfort          64
Live tracking         61
Name of passenger      1
Date of Review         4
Star Rating            0
Review Text            8
dtype: int64

In [11]:
# List of columns to fill with 0
cols_to_fill = [
    'Punctuality',
    'Cleanliness',
    'Staff Behaviour',
    'Driving',
    'Rest stop hygiene',
    'Seat comfort',
    'Live tracking',
    'AC'
]

# Fill missing values with 0 for each column in df_tg
for col in cols_to_fill:
    missing_before = df_tg[col].isna().sum()
    df_tg[col] = df_tg[col].fillna(0)
    missing_after = df_tg[col].isna().sum()
    print(f"{col}: Filled {missing_before} missing values → {missing_after} remaining")


Punctuality: Filled 17 missing values → 0 remaining
Cleanliness: Filled 15 missing values → 0 remaining
Staff Behaviour: Filled 43 missing values → 0 remaining
Driving: Filled 64 missing values → 0 remaining
Rest stop hygiene: Filled 58 missing values → 0 remaining
Seat comfort: Filled 64 missing values → 0 remaining
Live tracking: Filled 61 missing values → 0 remaining
AC: Filled 644 missing values → 0 remaining


In [12]:
df_tg['Name of passenger'] = df_tg['Name of passenger'].fillna('Unknown Passenger')
df_tg['Date of Review'] = df_tg['Date of Review'].fillna(pd.NaT)
df_tg['Review Text'] = df_tg['Review Text'].fillna("No Review Provided")


In [13]:
df_tg.isnull().sum()

Route                0
Bus Name             0
Bus Type             0
Price                0
Punctuality          0
Cleanliness          0
Staff Behaviour      0
Driving              0
AC                   0
Rest stop hygiene    0
Seat comfort         0
Live tracking        0
Name of passenger    0
Date of Review       4
Star Rating          0
Review Text          0
dtype: int64

In [14]:
df_tg.duplicated().sum()

np.int64(994)

In [15]:
df_tg = df_tg.drop_duplicates(keep='first')


In [16]:
df_tg.duplicated().sum()

np.int64(0)

In [17]:
df_tg.shape

(9020, 16)

In [18]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

# Sentiment score for df_tg
df_tg['Sentiment Score'] = df_tg['Review Text'].apply(
    lambda x: analyzer.polarity_scores(str(x))['compound']
)

# Function stays the same
def label_sentiment(score):
    if score >= 0.05:
        return 'Positive'
    elif score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# Apply sentiment label
df_tg['Sentiment Label'] = df_tg['Sentiment Score'].apply(label_sentiment)


In [19]:
df_tg['Route'] = df_tg['Route'].str.strip()
df_tg['Bus Name'] = df_tg['Bus Name'].str.strip()
df_tg['Bus Type'] = df_tg['Bus Type'].str.strip()
df_tg['Name of passenger'] = df_tg['Name of passenger'].str.strip()
df_tg['Review Text'] = df_tg['Review Text'].str.strip()


In [20]:
df_tg['review_id'] = range(1, len(df_tg) + 1)


In [21]:
df_tg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9020 entries, 0 to 10013
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Route              9020 non-null   object        
 1   Bus Name           9020 non-null   object        
 2   Bus Type           9020 non-null   object        
 3   Price              9020 non-null   int64         
 4   Punctuality        9020 non-null   float64       
 5   Cleanliness        9020 non-null   float64       
 6   Staff Behaviour    9020 non-null   float64       
 7   Driving            9020 non-null   float64       
 8   AC                 9020 non-null   float64       
 9   Rest stop hygiene  9020 non-null   float64       
 10  Seat comfort       9020 non-null   float64       
 11  Live tracking      9020 non-null   float64       
 12  Name of passenger  9020 non-null   object        
 13  Date of Review     9016 non-null   datetime64[ns]
 14  Star Rating 

In [22]:
cols_to_convert = [
    'Punctuality',
    'Cleanliness',
    'Staff Behaviour',
    'Driving',
    'AC',
    'Rest stop hygiene',
    'Seat comfort',
    'Live tracking'
]

df_tg[cols_to_convert] = df_tg[cols_to_convert].astype(int)


In [23]:
df_tg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9020 entries, 0 to 10013
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Route              9020 non-null   object        
 1   Bus Name           9020 non-null   object        
 2   Bus Type           9020 non-null   object        
 3   Price              9020 non-null   int64         
 4   Punctuality        9020 non-null   int64         
 5   Cleanliness        9020 non-null   int64         
 6   Staff Behaviour    9020 non-null   int64         
 7   Driving            9020 non-null   int64         
 8   AC                 9020 non-null   int64         
 9   Rest stop hygiene  9020 non-null   int64         
 10  Seat comfort       9020 non-null   int64         
 11  Live tracking      9020 non-null   int64         
 12  Name of passenger  9020 non-null   object        
 13  Date of Review     9016 non-null   datetime64[ns]
 14  Star Rating 

In [24]:
df_tg.duplicated().sum()

np.int64(0)

In [25]:
df_tg.isnull().sum()

Route                0
Bus Name             0
Bus Type             0
Price                0
Punctuality          0
Cleanliness          0
Staff Behaviour      0
Driving              0
AC                   0
Rest stop hygiene    0
Seat comfort         0
Live tracking        0
Name of passenger    0
Date of Review       4
Star Rating          0
Review Text          0
Sentiment Score      0
Sentiment Label      0
review_id            0
dtype: int64

In [26]:
df_tg.to_csv("redbus_data_cleaned_tgsrtc.csv", index=False)

In [27]:
import pymysql
import pandas as pd
import numpy as np


DB_HOST = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com"
DB_USER = "3843tbpvmio3tiw.root"
DB_PASSWORD = "BvhwR5z1itdG9CMJ"
DB_NAME = "redbus_reviews"          
DB_PORT = 4000
SSL_CA = r"D:\prakash\redbus_scraper\isrgrootx1 (3).pem"   
# --------------------------------------------------------------

# connect
connection = pymysql.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    database=DB_NAME,
    port=DB_PORT,
    ssl={"ca": SSL_CA},
    connect_timeout=600,
    autocommit=False,
    cursorclass=pymysql.cursors.Cursor
)
cursor = connection.cursor()

# Read your cleaned CSV
df = pd.read_csv(r"D:\prakash\redbus_scraper\redbus_data_cleaned_tgsrtc.csv", low_memory=False)


df = df.rename(columns=lambda c: c.strip().replace(' ', '_').replace('/', '_').replace('-', '_'))
df['Date_of_Review'] = pd.to_datetime(df['Date_of_Review'], errors='coerce')


# Create table SQL (adjust lengths if you want)
CREATE_TABLE = """
CREATE TABLE IF NOT EXISTS redbus_reviews_tgsrtc (
    review_id INT AUTO_INCREMENT PRIMARY KEY,
    Route VARCHAR(255),
    Bus_Name VARCHAR(255),
    Bus_Type VARCHAR(255),
    Price INT,
    Punctuality INT,
    Cleanliness INT,
    Staff_Behaviour INT,
    Driving INT,
    AC INT,
    Rest_stop_hygiene INT,
    Seat_comfort INT,
    Live_tracking INT,
    Name_of_passenger VARCHAR(255),
    Date_of_Review DATETIME,
    Star_Rating INT,
    Review_Text TEXT,
    Sentiment_Score FLOAT,
    Sentiment_Label VARCHAR(50)
) DEFAULT CHARSET=utf8mb4;
"""

# Create table
cursor.execute(CREATE_TABLE)
connection.commit()
print("Table created or already exists.")

# Prepare INSERT (use column order matching the table definition except review_id)
INSERT_QUERY = """
INSERT IGNORE INTO redbus_reviews_tgsrtc (
    Route, Bus_Name, Bus_Type, Price, Punctuality, Cleanliness, Staff_Behaviour, Driving,
    AC, Rest_stop_hygiene, Seat_comfort, Live_tracking, Name_of_passenger, Date_of_Review,
    Star_Rating, Review_Text, Sentiment_Score, Sentiment_Label
) VALUES (
    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
)
"""

# Convert DataFrame rows into tuples and handle NaT -> None
def row_to_tuple(row):
    # ensure same column names as used above (after rename)
    return (
        row.get('Route'),
        row.get('Bus_Name'),
        row.get('Bus_Type'),
        int(row['Price']) if not pd.isna(row['Price']) else None,
        int(row['Punctuality']) if not pd.isna(row['Punctuality']) else None,
        int(row['Cleanliness']) if not pd.isna(row['Cleanliness']) else None,
        int(row['Staff_Behaviour']) if not pd.isna(row['Staff_Behaviour']) else None,
        int(row['Driving']) if not pd.isna(row['Driving']) else None,
        int(row['AC']) if not pd.isna(row['AC']) else None,
        int(row['Rest_stop_hygiene']) if not pd.isna(row['Rest_stop_hygiene']) else None,
        int(row['Seat_comfort']) if not pd.isna(row['Seat_comfort']) else None,
        int(row['Live_tracking']) if not pd.isna(row['Live_tracking']) else None,
        row.get('Name_of_passenger'),
        None if pd.isna(row.get('Date_of_Review')) else row.get('Date_of_Review'),
        int(row['Star_Rating']) if not pd.isna(row['Star_Rating']) else None,
        row.get('Review_Text'),
        float(row['Sentiment_Score']) if not pd.isna(row['Sentiment_Score']) else None,
        row.get('Sentiment_Label')
    )

# Build list of tuples (use itertuples or apply)
tuples = []
for r in df.to_dict(orient='records'):
    tuples.append(row_to_tuple(r))

# Insert in batches
batch_size = 5000
for i in range(0, len(tuples), batch_size):
    batch = tuples[i:i+batch_size]
    try:
        cursor.executemany(INSERT_QUERY, batch)
        connection.commit()
        print(f"Inserted rows {i} to {i+len(batch)-1}")
    except Exception as e:
        connection.rollback()
        print("Error inserting batch:", e)
        raise

cursor.close()
connection.close()
print("Done.")


Table created or already exists.
Inserted rows 0 to 4999
Inserted rows 5000 to 9019
Done.
